In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

print('##############################')
print('数据预处理')
train = pd.read_csv('data/tap_fun_train.csv')
test = pd.read_csv('data/tap_fun_test.csv')
res0 = pd.DataFrame()
res0['user_id'] = test[test['pay_price']==0]['user_id']
res0['prediction_pay_price'] = 0


train = train[train['pay_price']>0]
test =test[test['pay_price']>0]




data = pd.concat([train,test],axis=0,ignore_index=True)

data['prediction_pay_price'].fillna(value=-1,inplace=True)


hour = []
register_date = []
register_day = []
for va in data['register_time'].values:
    hour.append(int(va.split(' ')[1].split(':')[0]))
    register_date.append(va.split(' ')[0])
    register_day.append(int(va.split(' ')[0].split('-')[2]))
data['register_hour'] = hour
data['register_date'] = register_date
data['register_day'] = register_day
#凌晨（0-6），上午（7-12），下午（13-18），晚上（19-23）
data['register_period'] = data['register_hour']//6

cols_value = []
for co in data.columns:
    if 'value' in co:
        cols_value.append(co)
n = len(cols_value)

for i in range(n//2):
    col_name = cols_value[2*i].replace('add','reserve')
    data[col_name] = data[cols_value[2*i]]-data[cols_value[2*i+1]]
    col_name = cols_value[2*i].replace('add_value','reduce_add_ratio')
    data[col_name] = data[cols_value[2*i+1]]/data[cols_value[2*i]]

data['sr_outpost_tier_level'] = data['sr_outpost_tier_2_level']+data['sr_outpost_tier_3_level']+data['sr_outpost_tier_4_level']

data['sr_outpost'] = data['sr_outpost_durability_level']*data['sr_outpost_tier_level']

data['sr_healing'] = data['sr_healing_space_level']*data['sr_healing_speed_level']



##统计特征
data['label'] = data['prediction_pay_price']/data['pay_price']
for co in data.columns:
    if 'level' in co or co in ['register_day','register_date','register_hour','register_period']:
        se = data[co].value_counts()
        data[co+'_cnt'] =  data[co].map(se)
        se = data.groupby(co)['pay_price'].mean()
        data[co+'_avg_pay'] = data[co].map(se)
        se = data.groupby(co)['avg_online_minutes'].mean()
        data[co+'_avg_online_minutes'] = data[co].map(se)
        se = data.groupby(co)['label'].mean()
        data[co+'_avg_label'] = data[co].map(se)
##week_day特征
register_date = data['register_date'].values.tolist()
se = pd.Series(pd.Series(register_date).drop_duplicates().sort_values().values)
se = pd.Series(se.index+5,index = se.values)%7
data['week_day'] = data['register_date'].map(se)
for co in ['register_time','register_date']:
    del data[co]
##pvp特征
data['pay_avg'] = data['pay_price']/data['pay_count']

data['pvp_win_battle_ratio'] = data['pvp_win_count']/data['pvp_battle_count']
data['pvp_lanch_battle_ratio'] = data['pvp_lanch_count']/data['pvp_battle_count']

data['pve_win_battle_ratio'] = data['pve_win_count']/data['pve_battle_count']
data['pve_lanch_battle_ratio'] = data['pve_lanch_count']/data['pve_battle_count']

data['pve_pvp_lanch_ratio'] = data['pve_lanch_count']/(data['pve_lanch_count']+data['pvp_lanch_count'])
data['pve_pvp_battle_ratio'] = data['pve_battle_count']/(data['pve_battle_count']+data['pvp_battle_count'])
data['pve_pvp_win_ratio'] = data['pve_win_count']/(data['pve_win_count']+data['pvp_win_count'])




for num in [10,20,30,50,70,100,200,500,1000,2000,5000,10000]:
    data['(pay_price)>'+str(num)] = (data['pay_price']>=num).astype(int)

train_x = data[data['prediction_pay_price']!=-1]
test_x = data[data['prediction_pay_price']==-1]
print(train_x.shape)
print (test_x.shape)


label = ['label','prediction_pay_price']
k = 0

res = pd.DataFrame()
res['user_id'] = test_x['user_id'].values
del train_x['user_id']
del test_x['user_id']
del train_x[label[1-k]]
del test_x[label[1-k]]
train_y = train_x.pop(label[k])
del test_x[label[k]]

lst = train_y.sort_values().index.tolist()
train_index = []
test_index = []
se = pd.Series(lst)
se = pd.Series(se.index,index=se.values)
for i in range(5):
    test_index.append(se[se%5==i].index.tolist())
    train_index.append(se[se%5!=i].index.tolist())



print('##############################')
print('开始训练')
print(train_x.shape)
print (test_x.shape)
best_params = {
        'num_leaves':10, 
        'learning_rate':0.01,

    'boosting':'gbdt',
    'min_child_samples':100,
    
    'feature_fraction_seed':0, 
    'feature_fraction':0.2,
    
    'bagging_freq':0,
    'bagging_seed':0,
    'bagging_fraction':1.0,
    
    'reg_alpha':2,
    'reg_lambda':2, 
    
    'metric':'rmse',
    'objective':'regression'
} 
train_weight = train_x['pay_price']

def get_predict_result(params,k,cols):
    df = pd.DataFrame()
    num = 0
    rmse = []
    type_feature = ['week_day','register_period','register_hour']
    for co in type_feature:
        if co not in cols:
            type_feature.remove(co)
    if len(type_feature)==0:
        type_feature='auto'
    for i in range(5):
        if k==1:
            train_part = lgb.Dataset(train_x[cols].loc[train_index[i]],weight=train_weight.loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = lgb.Dataset(train_x[cols].loc[test_index[i]],weight=train_weight.loc[test_index[i]],label=train_y.loc[test_index[i]])
        elif k==0:
            train_part = lgb.Dataset(train_x[cols].loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = lgb.Dataset(train_x[cols].loc[test_index[i]],label=train_y.loc[test_index[i]])
        bst = lgb.train(params,train_part, 
              num_boost_round=10000, valid_sets=[train_part,evals], 
              valid_names=['train','evals'], fobj=None,feval=None,
              categorical_feature=type_feature,
                        early_stopping_rounds=50,
              evals_result=None, verbose_eval=50, learning_rates=None, 
              keep_training_booster=False, callbacks=None)
        num+=1
        test_ypre = bst.predict(test_x[cols])
        se = pd.Series(bst.feature_importance(),index=train_x.columns).sort_values(ascending=False)
        print(se.head())
        df['pre_'+str(num)]=test_ypre
        rmse.append(bst.best_score['evals']['rmse'])
    print(rmse)
    print(np.array(rmse).mean())
    return df,rmse
print('##############################')
print('lgb_1模型')
col_all = train_x.columns.tolist()
result,rmse = get_predict_result(best_params,1,col_all)
se = result.mean(1)
result['prediction_pay_price'] = se.values*test_x['pay_price'].values
result['user_id'] = res['user_id']
sub1 = result[['user_id','prediction_pay_price']]
sub1 = pd.concat([sub1,res0],axis=0,ignore_index=True)
score = round(np.array(rmse).mean(),3)
filename = 'result/lgb_1.csv'
sub1.to_csv(filename,index=False)
print(filename)

print('##############################')
print('lgb_2模型')

def get_predict_result(params,k,cols):
    df = pd.DataFrame()
    num = 0
    rmse = []
    type_feature = ['week_day','register_period','register_hour']
    for co in type_feature:
        if co not in cols:
            cols.append(co)
    for i in range(5):
        if k==1:
            train_part = lgb.Dataset(train_x[cols].loc[train_index[i]],weight=train_weight.loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = lgb.Dataset(train_x[cols].loc[test_index[i]],weight=train_weight.loc[test_index[i]],label=train_y.loc[test_index[i]])
        elif k==0:
            train_part = lgb.Dataset(train_x[cols].loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = lgb.Dataset(train_x[cols].loc[test_index[i]],label=train_y.loc[test_index[i]])
        bst = lgb.train(params,train_part, 
              num_boost_round=10000, valid_sets=[train_part,evals], 
              valid_names=['train','evals'], fobj=None,feval=None,
              categorical_feature=type_feature,
                        early_stopping_rounds=50,
              evals_result=None, verbose_eval=50, learning_rates=None, 
              keep_training_booster=False, callbacks=None)
        num+=1
        test_ypre = bst.predict(test_x[cols])
        se = pd.Series(bst.feature_importance(),index=cols).sort_values(ascending=False)
        print(se.head())
        df['pre_'+str(num)]=test_ypre
        rmse.append(bst.best_score['evals']['rmse'])
    print(rmse)
    print(np.array(rmse).mean())
    return df,rmse
def getCols(params,k=1):
    se = pd.Series(0,index=train_x.columns)
    for i in range(5):
        if k==1:
            train_part = lgb.Dataset(train_x.loc[train_index[i]],weight=train_weight.loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = lgb.Dataset(train_x.loc[test_index[i]],weight=train_weight.loc[test_index[i]],label=train_y.loc[test_index[i]])
        elif k==0:
            train_part = lgb.Dataset(train_x.loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = lgb.Dataset(train_x.loc[test_index[i]],label=train_y.loc[test_index[i]])
        bst = lgb.train(params,train_part, 
              num_boost_round=10000, valid_sets=[train_part,evals], 
              valid_names=['train','evals'], fobj=None,feval=None,
              categorical_feature=['week_day','register_period','register_hour'],
                        early_stopping_rounds=50,
              evals_result=None, verbose_eval=50, learning_rates=None, 
              keep_training_booster=False, callbacks=None)
        se = se+pd.Series(bst.feature_importance(),index=train_x.columns)
    se = se.sort_values(ascending=False)
    return se
col_all = getCols(best_params,k=1).index.tolist()

col_all = getCols(best_params,k=1).index.tolist()
result,rmse = get_predict_result(best_params,1,col_all[:50])
se = result.mean(1)
result['prediction_pay_price'] = se.values*test_x['pay_price'].values
result['user_id'] = res['user_id']
sub1 = result[['user_id','prediction_pay_price']]
sub1 = pd.concat([sub1,res0],axis=0,ignore_index=True)
score = round(np.array(rmse).mean(),3)
filename = 'result/lgb_2.csv'
sub1.to_csv(filename,index=False)
print(filename)
print('Done')

##############################
数据预处理
(41439, 448)
(19549, 448)
##############################
开始训练
(41439, 445)
(19549, 445)
##############################
lgb_1模型
Training until validation scores don't improve for 50 rounds.
[50]	train's rmse: 7.63327	evals's rmse: 8.37862
[100]	train's rmse: 7.5802	evals's rmse: 8.36593
[150]	train's rmse: 7.53417	evals's rmse: 8.359
[200]	train's rmse: 7.49348	evals's rmse: 8.35607
[250]	train's rmse: 7.45863	evals's rmse: 8.35358
[300]	train's rmse: 7.427	evals's rmse: 8.35212
[350]	train's rmse: 7.39746	evals's rmse: 8.34897
[400]	train's rmse: 7.36943	evals's rmse: 8.34172
[450]	train's rmse: 7.3423	evals's rmse: 8.34058
[500]	train's rmse: 7.31744	evals's rmse: 8.341
[550]	train's rmse: 7.29571	evals's rmse: 8.33976
Early stopping, best iteration is:
[544]	train's rmse: 7.29833	evals's rmse: 8.33914
register_hour             276
avg_online_minutes        136
pay_price                  87
pay_avg                    86
stone_reduce_add_ratio     8